In [1]:
import findspark
findspark.init('C:\\spark\spark-3.2.1-bin-hadoop3.2')  
import pyspark
from pyspark.sql import SparkSession 
from pyspark import SparkContext, SQLContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql.types import * 
from pyspark.sql.functions import *
from  datetime import datetime
import os
import shutil
import time


checkpointDir = "./CheckPoint"
path = "./result"

# очищаем директорию чекпоинтов
try:
    shutil.rmtree(checkpointDir)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))


# очищаем директорию чекпоинтов пути сохранения
try:
    shutil.rmtree(path)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror)) 

csvSchema = "symbol String, price Float"
    
spark = (SparkSession
 .builder
 .appName('struct_stream_4')
 .enableHiveSupport()
 .getOrCreate())

# Открытие потока на чтение
crypto_currency = (spark
  .readStream
  .format("csv")
  .option("delimiter", ";")
  .schema(csvSchema)
  .load("./source"))

# Добавляю столбец с датой и временем прихода данных
crypto_currency = crypto_currency \
    .withColumn('sampling_datetime', lit(datetime.now().strftime("%Y-%m-%d %H:%M:%S")))


# Запись измененного потока в директорию в формате json
streamingQuery = (new_crypto_currency             
    .repartition(1)    
    .writeStream
    .outputMode("append")
    .format("json")
    .option("checkpointLocation", checkpointDir) 
    .trigger(processingTime="5 second")
    .start(path))

streamingQuery.awaitTermination()    

Error: ./CheckPoint - Системе не удается найти указанный путь.
Error: ./result - Системе не удается найти указанный путь.


Py4JJavaError: An error occurred while calling o37.load.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1215)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1420)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:225)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(Trave

In [2]:
from  datetime import datetime
datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2022-02-20 12:06:29'

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, HiveContext


sparkSession = (SparkSession
                .builder
                .appName('example-pyspark-read-and-write-from-hive')
                .config("hive.metastore.uris", "thrift://localhost:9083", conf=SparkConf())
                .enableHiveSupport()
                .getOrCreate()
                )

# Write into Hive
#df.write.saveAsTable('example')

df_load = sparkSession.sql('SELECT * FROM example')
df_load.show()
print(df_load.show())